<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Logo_uninorte_colombia.jpg/1200px-Logo_uninorte_colombia.jpg" width=70%></img>

<font size=12>Algoritmos Y Complejidad</font>\
<font size=6>Algoritmos Geneticos: Problema de las plantas</font>

Héctor Andrés Suárez Camacho - 200162159 \
Fernando Mateo Valencia Gómez - 200166289\
Santiago Carbal Martinez - 200152335\
Santiago Hernández Cadavid - 200153162


In [92]:
import random
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()

Un diccionario para guardar los posibles valores de cada gen

In [93]:
# Definir los posibles valores de cada gen
genes = {
    'resistencia_humedad': ['A', 'B'],
    'resistencia_calor': ['A', 'B'],
    'numero_hojas': ['A', 'B'],
    'altura_tallo': ['A', 'B']
}

Esta función calcula el fitness de cada individuo, "bit por bit" para comprobar que tan bueno es comparandolo con el cromosoma objetivo:\

por ejemplo, el mejor cromosoma es AAAB, y tiene fitness de 8.


* Gen de resistencia a la humedad, si es A se otorga 2
*   Gen de resistencia al calor, si es A se otorga 2
*   Numero de hojas, si es A se otorga 3
* Altura del tallo, si es B se otorga 1





In [94]:
# Definir el cromosoma objetivo
objetivo = 'AAAB'

# Definir la función de aptitud para evaluar la calidad de cada individuo
def aptitud(cromosoma):
    puntaje = 0
    for i in range(len(objetivo)):
        if cromosoma[i] == objetivo[i]:
            if i == 0 or i == 1:
                puntaje += 2
            elif i == 2:
                puntaje += 3
            else:
                puntaje += 1
    return puntaje

Esta función pone a competir dos individuos de la población para escoger a los de mejor fitness. Escoge de a dos y los compara, el de mejor fitness se va para la lista de selección y cuando el torneo ya ha terminado, los de la lista de selección pasan a ser la nueva generación.

In [95]:
# Definir la función de selección por torneo
def seleccion_torneo(poblacion, n):
    seleccionados = []
    recuento = {}
    for individuo in poblacion:
        recuento[individuo] = 0

    for i in range(n):
        individuos = random.sample(poblacion, 2)
        aptitud1 = aptitud(individuos[0])
        aptitud2 = aptitud(individuos[1])
        if aptitud1 > aptitud2:
            seleccionados.append(individuos[0])
            recuento[individuos[0]] += 1
        else:
            seleccionados.append(individuos[1])
            recuento[individuos[1]] += 1
    return seleccionados, recuento

En esta función de cruce, divide a los cromosomas padres por el punto de cruce y los combina para formar hijos:


*   Punto de cruce: 2
*   Padre 1: AA|BA
*   PAdre 2: BB|AA

*   Hijo 1: AAAA
*   hijo 2: BBBA




In [96]:
# Definir la función de cruce en un punto
def cruce(individuo1, individuo2, crossover_point):
    hijo1 = individuo1[:crossover_point] + individuo2[crossover_point:]  # subcadena[inicio:final:saltos]
    hijo2 = individuo2[:crossover_point] + individuo1[crossover_point:]
    return hijo1, hijo2

La mutación altera un gen de un cromosoma con el proposito de añadir diversidad a las poblaciones. Esto se hace con el proposito de que intente encontrar una solución optma en otra parte del espacio muestral.

En este caso: Se recorre cada gen del cromosoma y se saca un número aleatoria, si esta dentro del 0.1 de probabilidades de mutar, mutará.

ejemplo: Tenemos AAAA, y recorriendo los genes individualmente, el ultimo gen A muta a B, por lo tanto quedaría AAAB

In [97]:
# Definir la función de mutación
def mutacion(individuo):
    indice = random.randint(0, len(objetivo)-1)
    nuevo_valor = random.choice(genes[list(genes.keys())[indice]])
    individuo_mutado = individuo[:indice] + nuevo_valor + individuo[indice+1:]
    return individuo_mutado

Aquí de definen los parámetros del algoritmo genetico. \

Se pasa la población inicial, el número de generaciones que se van a ejecutar, el tamaño del torneo para indicar de a cuantos individuos se van a comparar en el torneo, en este caso de a pares. Y por ultimo las probabilidades de emparejamiento y mutación.

In [98]:
# Definir los parámetros del algoritmo genético
poblacion = ['BAAA', 'BBBB', 'ABBA', 'BABA']  # Población inicial
num_generaciones = 2  # Número de generaciones a simular
tam_torneo = 2  # Tamaño del torneo para la selección
pe = 0.7  # Probabilidad de emparejamiento
pm = 0.1  # Probabilidad de mutación

Se crea el dataframe para visualizar la salida de una mejor manera

In [99]:
# Crear un DataFrame para almacenar los resultados
columnas = ['Generación', 'Población', 'Fitness', 'Probabilities', 'Count', 'Mating Pool', 'Crossover Point', 'Offspring After Xover', 'Fitness Post Xover', 'Mutation', 'Mutation Chromosome','Fitness Post Mutation' ]
resultados = pd.DataFrame(columns=columnas)


Como funciona la solución:

* Cada iteración del bucle for representa una generación
* En cada generación se evalua el fitness de cada individuo en la población actual.
* Luego se realiza un torneo para elegir a las parejas de padres que van a cruzarse, es decir, se escogen a los mejores individuos y se cruzan.
* Una vez se tiene la mating pool, se calculan los puntos de cruce para cada pareja.
* Luego se hace el cruce y se obtienen los hijos; dos hijos por cada pareja.
* Luego se calcula el fitnes de los nuevos individuos (hijos).
* Luego se revisa cada gen de cada hijo y de acuerdo a la probabilidad de mutación, mutará el gen o no
* Si muta, vuelve a calcular el fitness de la población
*Esa sería una generación y se repiten los pasos.

In [100]:

# Ejecutar el algoritmo genético
for generacion in range(num_generaciones):
    # Evaluar la aptitud de cada individuo
    aptitudes = [aptitud(individuo) for individuo in poblacion]

    # Seleccionar la nueva población mediante el torneo
    seleccionados, recuento = seleccion_torneo(poblacion, len(poblacion))

    # Calcular la aptitud total y las probabilidades de selección
    total_aptitud = sum(aptitudes)
    probabilidades = [aptitud / total_aptitud for aptitud in aptitudes]

    # Crear el DataFrame de la generación actual
    generacion_actual = pd.DataFrame(columns=columnas)

    # Agregar los datos a la tabla de resultados de la generación actual
    generacion_actual['Generación'] = [generacion + 1] * len(poblacion)
    generacion_actual['Población'] = poblacion
    generacion_actual['Fitness'] = aptitudes
    generacion_actual['Probabilities'] = probabilidades
    generacion_actual['Count'] = [recuento[individuo] for individuo in poblacion]

    # Seleccionar la mating pool
    mating_pool = seleccionados

    # Ajustar la longitud de la mating pool si es necesario
    if len(mating_pool) != len(poblacion):
        mating_pool = pd.Series(mating_pool)  # Convertir a una Serie de pandas
        generacion_actual['Mating Pool'] = mating_pool
    else:
        generacion_actual['Mating Pool'] = mating_pool

    # Llenar los valores faltantes con NaN
    generacion_actual['Mating Pool'] = generacion_actual['Mating Pool'].fillna(np.nan)

    # Actualizar los valores faltantes con los valores correspondientes
    for i in range(len(generacion_actual)):
        if pd.isnull(generacion_actual.loc[i, 'Mating Pool']):
            generacion_actual.loc[i, 'Mating Pool'] = random.choice(mating_pool)

    # Realizar el cruce en un punto
    offspring = []
    crossover_points = [random.randint(1, len(objetivo)-1) for _ in range(len(mating_pool)//2)]
    for i in range(0, len(mating_pool), 2):
        pair = mating_pool[i:i+2]
        crossover_point = crossover_points[i//2]  # Obtener el crossover point correspondiente a la pareja actual
        if random.random() < pe:
            hijo1, hijo2 = cruce(pair[0], pair[1], crossover_point)
            offspring.append(hijo1)
            offspring.append(hijo2)
        else:
            offspring.append(pair[0])
            offspring.append(pair[1])

    # Ajustar la longitud de los offspring para que coincida con la población
    offspring = offspring[:len(poblacion)]

    # Agregar los datos de los offspring después del cruce a la tabla de resultados
    generacion_actual['Crossover Point'] = crossover_points * 2  # Repetir los valores de crossover_points para cada pareja
    generacion_actual['Offspring After Xover'] = offspring
    generacion_actual['Fitness Post Xover'] = [aptitud(individuo) for individuo in offspring]

    # Realizar la mutación
    nuevos_cromosomas = []
    for i in range(len(offspring)):
        if random.random() < pm:
            mutado = mutacion(offspring[i])
            nuevos_cromosomas.append(mutado)
            offspring[i] = mutado
        else:
            nuevos_cromosomas.append(offspring[i])

    # Agregar los datos de la mutación a la tabla de resultados
    generacion_actual['Mutation'] = [True if offspring[i] != generacion_actual['Offspring After Xover'][i] else False for i in range(len(offspring))]
    generacion_actual['Mutation Chromosome'] = nuevos_cromosomas
    generacion_actual['Fitness Post Mutation'] = [aptitud(individuo) for individuo in offspring]


    # Actualizar la población
    poblacion = offspring

    # Agregar la generación actual a los resultados
    resultados = pd.concat([resultados, generacion_actual], ignore_index=True)

    # Mostrar la generación actual
    print(f"Generación {generacion + 1}:")
    display(data_table.DataTable(resultados[resultados['Generación'] == generacion+1], include_index=False))
    print("----------------------------------------")


Generación 1:


,Generación,Población,Fitness,Probabilities,Count,Mating Pool,Crossover Point,Offspring After Xover,Fitness Post Xover,Mutation,Mutation Chromosome,Fitness Post Mutation
0,1,BAAA,5,0.5,2,BABA,2,BABA,2,False,BABA,2
1,1,BBBB,1,0.1,0,ABBA,3,ABBA,2,False,ABBA,2
2,1,ABBA,2,0.2,1,BAAA,2,BAAA,5,False,BAAA,5
3,1,BABA,2,0.2,1,BAAA,3,BAAA,5,False,BAAA,5


----------------------------------------
Generación 2:


,Generación,Población,Fitness,Probabilities,Count,Mating Pool,Crossover Point,Offspring After Xover,Fitness Post Xover,Mutation,Mutation Chromosome,Fitness Post Mutation
4,2,BABA,2,0.142857,1,BAAA,2,BAAA,5,False,BAAA,5
5,2,ABBA,2,0.142857,0,BAAA,2,BAAA,5,False,BAAA,5
6,2,BAAA,5,0.357143,3,BABA,2,BAAA,5,False,BAAA,5
7,2,BAAA,5,0.357143,3,BAAA,2,BABA,2,False,BABA,2


----------------------------------------
